In [ ]:
!pip install git+https://github.com/suno-ai/bark.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-2h2rifjf
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-2h2rifjf
  Resolved https://github.com/suno-ai/bark.git to commit f6f2db527b13c4a3e52ed6fbac587aadc3723eb6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 92.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 84.7 MB/s e

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from bark.generation import load_codec_model, generate_text_semantic
from encodec.utils import convert_audio

import torchaudio
import torch

model = load_codec_model(use_gpu=True)

Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:02<00:00, 37.2MB/s]


In [ ]:
# Load and pre-process the audio waveform
audio_filepath = '/content/drive/MyDrive/cs_172b_project/serpAI_test2.wav' # the audio you want to clone (will get truncated so 5-10 seconds is probably fine, existing samples that I checked are around 7 seconds)
device = 'cuda' # or 'cpu'
#device = 'cpu'
wav, sr = torchaudio.load(audio_filepath)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.unsqueeze(0).to(device)

In [ ]:
# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav)
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()  # [n_q, T]

In [ ]:
#transcription of the audio you are cloning
text = "Says derivative is negative. So you're going to move like this, and you're going to end up to this fixed point here."

In [ ]:
# get seconds of audio
seconds = wav.shape[-1] / model.sample_rate
# generate semantic tokens
semantic_tokens = generate_text_semantic(text, max_gen_duration_s=seconds, top_k=50, top_p=.95, temp=0.7) # not 100% 

100%|██████████| 100/100 [00:21<00:00,  4.76it/s]


In [ ]:
# move codes to cpu
codes = codes.cpu().numpy()

In [ ]:
import numpy as np
voice_name = 'pierre' # whatever you want the name of the voice to be
output_path = '/content/drive/MyDrive/cs_172b_project/' + voice_name + '.npz'
np.savez(output_path, fine_prompt=codes, coarse_prompt=codes[:2, :], semantic_prompt=semantic_tokens)

In [ ]:
#generate voice
from bark.api import generate_audio
from transformers import BertTokenizer
from bark.generation import SAMPLE_RATE, preload_models, codec_decode, generate_coarse, generate_fine, generate_text_semantic

# Enter your prompt and speaker here
text_prompt = "Tractors for farming are used for plowing, cultivating the soil, and planting fields. Modern types of tractors can be used for a variety of tasks including shrubs, landscaping, moving or spreading fertilizer, and lawn maintenance."
voice_name = "pierre" # use your custom voice name here if you have one

# load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

In [ ]:
# download and load all models
preload_models(
    text_use_gpu=True,
    text_use_small=False,
    coarse_use_gpu=True,
    coarse_use_small=False,
    fine_use_gpu=True,
    fine_use_small=False,
    codec_use_gpu=True,
    force_reload=False#,
    #path="models"
)

In [ ]:
# simple generation
voice_path = '/content/drive/MyDrive/cs_172b_project/pierre.npz'
audio_array = generate_audio(text_prompt, history_prompt=voice_path, text_temp=0.7, waveform_temp=0.7)

100%|██████████| 36/36 [00:33<00:00,  1.06it/s]


In [ ]:
# generation with more control
x_semantic = generate_text_semantic(
    text_prompt,
    history_prompt=voice_path,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)

x_coarse_gen = generate_coarse(
    x_semantic,
    history_prompt=voice_path,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)
x_fine_gen = generate_fine(
    x_coarse_gen,
    history_prompt=voice_path,
    temp=0.5,
)
audio_array = codec_decode(x_fine_gen)

100%|██████████| 36/36 [07:57<00:00, 13.25s/it]


In [ ]:
from IPython.display import Audio

print(text_prompt)

# play audio
Audio(audio_array, rate=SAMPLE_RATE)

Tractors for farming are used for plowing, cultivating the soil, and planting fields. Modern types of tractors can be used for a variety of tasks including shrubs, landscaping, moving or spreading fertilizer, and lawn maintenance.


In [ ]:
from scipy.io.wavfile import write as write_wav

# save audio
filepath = "/content/drive/MyDrive/cs_172b_project/serpAI_test_audio2.wav" # change this to your desired output path
write_wav(filepath, SAMPLE_RATE, audio_array)